In [2]:
from model.setup import DataLoader
from model.algo import *
import model.config as config

C:\Users\657\657_paper\volatility\myself_paper_vol
C:\Users\657\657_paper\volatility\myself_paper_vol\thesis_code\5_HiTec


In [3]:
industry_ = str(config.industry)[-5:]
df = pd.read_parquet(str(config.path / f"thesis_code/portfolios_data/industry_{industry_}.parquet"))
df.index = pd.to_datetime(df.index)  # 轉格式為日期格式
dataloader = DataLoader(start='1980-01-01', end='2021-12-01')
df['MthCap'] = df['MthCap'].pct_change()
df = dataloader.get_macro(df)
df = df['1990':'2021'].dropna(axis=1)  # 要求1990-2021年都要有資料的變數
print(df.shape)

(384, 414)


In [4]:
sector_dict = dataloader.get_categories(df)
financial = sector_dict['f']
operating = sector_dict['o']
technical = sector_dict['p']
supply = sector_dict['sup']
firm_senti = sector_dict['sen']
esg = sector_dict['esg']
output_income = sector_dict['1']
labor = sector_dict['2']
housing = sector_dict['3']
consump = sector_dict['4']
money = sector_dict['5']
intExRate = sector_dict['6']
prices = sector_dict['7']
stock = sector_dict['8']
market_senti = sector_dict['9']

sector_list = [financial, operating, technical, supply, firm_senti, esg, output_income, labor, housing, consump, money,
               intExRate, prices, stock, market_senti]
keep_list = [financial.copy(), operating.copy(), technical.copy(), supply.copy(), firm_senti.copy(), esg.copy(),
             output_income.copy(), labor.copy(),
             housing.copy(), consump.copy(), money.copy(), intExRate.copy(), prices.copy(), stock.copy(),
             market_senti.copy()]

print(len(technical) + len(financial) + len(operating) + len(supply) + len(firm_senti) + len(esg)
      + len(output_income) + len(labor) + len(housing) + len(consump) + len(money) + len(intExRate) + len(prices) + len(
    stock) + len(market_senti))

414


In [5]:
stat_df = pd.DataFrame(columns=[1, 2, 3, 4, 5, 6, 7, 15],
                       index=['PCA-QR(0.1)', 'PCA-QR(0.5)', 'PCA-QR(0.9)',
                              'PLS-QR(0.1)', 'PLS-QR(0.5)', 'PLS-QR(0.9)',
                              'PQR-QR(0.1)', 'PQR-QR(0.5)', 'PQR-QR(0.9)',
                              'PCA-QR(0.1)*', 'PCA-QR(0.5)*', 'PCA-QR(0.9)*',
                              'PLS-QR(0.1)*', 'PLS-QR(0.5)*', 'PLS-QR(0.9)*',
                              'PQR-QR(0.1)*', 'PQR-QR(0.5)*', 'PQR-QR(0.9)*'])

In [31]:
# HiTec PCA 上行(0.9) 4 因子
stat_var_df = pd.DataFrame(columns=['var', 'Marginal_PseudoR'])
stat_cat_df = pd.DataFrame(columns=['cat', 'Marginal_PseudoR'])
#區分四類別
funda = financial + operating + supply + esg
macro = output_income + labor + housing + consump + money + intExRate + prices + stock
senti = firm_senti + market_senti
sector_list = [funda, technical, macro, senti]
keep_list = [funda.copy(), technical.copy(), macro.copy(), senti.copy()]

oos_periods = len(df) - 120
q = 0.9

ols = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
ols.seperate(sector_list, dr_algo='fit_pca', n_components=1, oos_periods=oos_periods)
ols.fit_recursive(quantile=q)
pred_df, stat = ols.statistics_recursive()
print(f'PCA_(sep4)-QR({q}):', round(stat['out of sample'][1], 4))

s1 = []
for i in range(len(sector_list)):
    s1 += [i] * len(sector_list[i])
s2 = []
for i in range(len(sector_list)):
    s2 += range(len(sector_list[i]))
# p.34 圖14,15
for x, y in zip(s1, s2):
    var = sector_list[x][y]
    sector_list2 = sector_list[:x] + [sector_list[x][:y] + sector_list[x][y + 1:]] + sector_list[
                                                                                     x + 1:]  # 用切片做，移除sector_list[x][y]，不能用del或remove，會完全改變。
    ols2 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols2.seperate(sector_list2, dr_algo='fit_pca', n_components=1, oos_periods=oos_periods)
    ols2.fit_recursive(quantile=q)
    pred_df2, stat2 = ols2.statistics_recursive()
    print(var, ':', round(stat['out of sample'][1] - stat2['out of sample'][1], 4))
    stat_var_df = stat_var_df._append(
        {'var': var, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat2['out of sample'][1], 4)},
        ignore_index=True)
    if stat['out of sample'][1] - stat2['out of sample'][1] <= 0:
        keep_list[x].remove(var)
total_ = 0
for i in range(len(keep_list)):
    total_ += len(keep_list[i])
print(f'keep_list有{total_}個關鍵變數')  # 53個沒錯!!!
print('keep_list=', keep_list)

# p.36 圖17 類別的邊際貢獻: stat3-stat5  用切片做 沒錯!!!
for i in range(len(sector_list)):
    sector_list3 = sector_list[:i] + sector_list[i + 1:]  # keep_list刪掉就回不去了 用切片做
    ols5 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols5.seperate(sector_list3, dr_algo='fit_pca', n_components=1,
                  oos_periods=oos_periods)  # n_components=1 沒錯 從sector_list3的11類別 個別萃取出1個因子，共11個
    ols5.fit_recursive(quantile=q)
    pred_df5, stat5 = ols5.statistics_recursive()
    #print(stat5)
    print(i, ': Marginal PsuedoR2=', round(stat['out of sample'][1] - stat5['out of sample'][1], 4))
    print('----------------------------------------------------------------')
    stat_cat_df = stat_cat_df._append(
        {'cat': i, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat5['out of sample'][1], 4)},
        ignore_index=True)

print(stat_var_df)
print(stat_cat_df)
stat_var_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PCA_UP_4_KeyVar.csv"))
stat_cat_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PCA_UP_4_KeyCat.csv"))

PCA_(sep4)-QR(0.9): 0.0414
MthCap : 0.0034
CAPEI : 0.0088
bm : 0.0033
evm : 0.0005
pe_op_basic : 0.013
pe_op_dil : 0.0055
pe_exi : 0.008
pe_inc : 0.0042
ps : 0.0016
pcf : 0.0075
dpr : 0.003
cfm : -0.0016
equity_invcap : 0.0005
debt_invcap : 0.0007
totdebt_invcap : 0.0039
capital_ratio : 0.0004
int_debt : 0.0047
int_totdebt : -0.0026
cash_lt : -0.0038
invt_act : 0.0063
rect_act : -0.0038
debt_at : 0.0039
debt_ebitda : 0.004
short_debt : 0.0007
curr_debt : 0.0068
lt_debt : 0.0046
profit_lct : 0.0032
ocf_lct : 0.0051
cash_debt : 0.0103
fcf_ocf : -0.0016
lt_ppent : -0.003
dltt_be : 0.0004
debt_assets : 0.0069
debt_capital : 0.0063
de_ratio : 0.0084
intcov : 0.0003
intcov_ratio : 0.0004
ptb : 0.0026
PEG_trailing : 0.0001
divyield : -0.0049
AM : -0.0019
AssetGrowth : -0.0029
BM : 0.0077
BMdec : 0.0021
BookLeverage : 0.0073
BPEBM : 0.0018
Cash : 0.0002
CF : 0.0012
cfp : -0.0007
ChEQ : 0.0041
ChTax : 0.0023
CompEquIss : 0.0115
CompositeDebtIssuance : -0.0008
ConvDebt : 0.0024
DebtIssuance : -0

In [32]:
# HiTec PCA 上行(0.9) 15 因子
stat_var_df = pd.DataFrame(columns=['var', 'Marginal_PseudoR'])
stat_cat_df = pd.DataFrame(columns=['cat', 'Marginal_PseudoR'])
#區分15類別
sector_list = [financial, supply, firm_senti, technical, operating, esg, output_income, labor, housing, consump, money,
               intExRate, prices, stock, market_senti]
keep_list = [financial.copy(), supply.copy(), firm_senti.copy(), technical.copy(), operating.copy(), esg.copy(),
             output_income.copy(), labor.copy(), housing.copy(), consump.copy(), money.copy(), intExRate.copy(),
             prices.copy(), stock.copy(), market_senti.copy()]

oos_periods = len(df) - 120
q = 0.9

ols = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
ols.seperate(sector_list, dr_algo='fit_pca', n_components=1, oos_periods=oos_periods)
ols.fit_recursive(quantile=q)
pred_df, stat = ols.statistics_recursive()
print(f'PCA_(sep15)-QR({q}):', round(stat['out of sample'][1], 4))

s1 = []
for i in range(len(sector_list)):
    s1 += [i] * len(sector_list[i])
s2 = []
for i in range(len(sector_list)):
    s2 += range(len(sector_list[i]))
# p.34 圖14,15
for x, y in zip(s1, s2):
    var = sector_list[x][y]
    sector_list2 = sector_list[:x] + [sector_list[x][:y] + sector_list[x][y + 1:]] + sector_list[
                                                                                     x + 1:]  # 用切片做，移除sector_list[x][y]，不能用del或remove，會完全改變。
    ols2 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols2.seperate(sector_list2, dr_algo='fit_pca', n_components=1, oos_periods=oos_periods)
    ols2.fit_recursive(quantile=q)
    pred_df2, stat2 = ols2.statistics_recursive()
    print(var, ':', round(stat['out of sample'][1] - stat2['out of sample'][1], 4))
    stat_var_df = stat_var_df._append(
        {'var': var, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat2['out of sample'][1], 4)},
        ignore_index=True)
    if stat['out of sample'][1] - stat2['out of sample'][1] <= 0:
        keep_list[x].remove(var)
total_ = 0
for i in range(len(keep_list)):
    total_ += len(keep_list[i])
print(f'keep_list有{total_}個關鍵變數')  # 53個沒錯!!!
print('keep_list=', keep_list)

# p.36 圖17 類別的邊際貢獻: stat3-stat5  用切片做 沒錯!!!
for i in range(len(sector_list)):
    sector_list3 = sector_list[:i] + sector_list[i + 1:]  # keep_list刪掉就回不去了 用切片做
    ols5 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols5.seperate(sector_list3, dr_algo='fit_pca', n_components=1,
                  oos_periods=oos_periods)  # n_components=1 沒錯 從sector_list3的11類別 個別萃取出1個因子，共11個
    ols5.fit_recursive(quantile=q)
    pred_df5, stat5 = ols5.statistics_recursive()
    #print(stat5)
    print(i, ': Marginal PsuedoR2=', round(stat['out of sample'][1] - stat5['out of sample'][1], 4))
    print('----------------------------------------------------------------')
    stat_cat_df = stat_cat_df._append(
        {'cat': i, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat5['out of sample'][1], 4)},
        ignore_index=True)

print(stat_var_df)
print(stat_cat_df)
stat_var_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PCA_UP_15_KeyVar.csv"))
stat_cat_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PCA_UP_15_KeyCat.csv"))

PCA_(sep15)-QR(0.9): -0.018
MthCap : 0.0001
CAPEI : 0.0066
bm : 0.02
evm : 0.0128
pe_op_basic : 0.0049
pe_op_dil : 0.0071
pe_exi : 0.0025
pe_inc : -0.0015
ps : 0.0031
pcf : 0.0073
dpr : -0.0003
cfm : 0.002
equity_invcap : -0.0014
debt_invcap : 0.0024
totdebt_invcap : 0.0027
capital_ratio : 0.0058
int_debt : 0.0023
int_totdebt : 0.0013
cash_lt : 0.0137
invt_act : 0.0014
rect_act : -0.0022
debt_at : -0.0011
debt_ebitda : 0.0021
short_debt : -0.0003
curr_debt : -0.0005
lt_debt : -0.0011
profit_lct : 0.0022
ocf_lct : 0.0005
cash_debt : 0.0004
fcf_ocf : -0.0033
lt_ppent : -0.0026
dltt_be : 0.0245
debt_assets : 0.0041
debt_capital : -0.0019
de_ratio : 0.0001
intcov : 0.0027
intcov_ratio : 0.0015
ptb : 0.0036
PEG_trailing : -0.0014
divyield : 0.0177
AM : 0.0177
AssetGrowth : 0.0002
BM : -0.0016
BMdec : 0.0012
BookLeverage : 0.0065
BPEBM : 0.0008
Cash : -0.0013
CF : 0.003
cfp : 0.0147
ChEQ : 0.0011
ChTax : -0.0023
CompEquIss : 0.0038
CompositeDebtIssuance : 0.0011
ConvDebt : 0.0025
DebtIssuanc

In [33]:
# HiTec PLS 上行(0.9) 6 因子
stat_var_df = pd.DataFrame(columns=['var', 'Marginal_PseudoR'])
stat_cat_df = pd.DataFrame(columns=['cat', 'Marginal_PseudoR'])

#區分6類別
macro = output_income + labor + housing + consump + money + intExRate + prices + stock
senti = firm_senti + market_senti
opera = operating + supply
sector_list = [financial, opera, technical, esg, macro, senti]
keep_list = [financial.copy(), opera.copy(), technical.copy(), esg.copy(), macro.copy(), senti.copy()]

oos_periods = len(df) - 120
q = 0.9

ols = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
ols.seperate(sector_list, dr_algo='fit_pls', n_components=1, oos_periods=oos_periods)
ols.fit_recursive(quantile=q)
pred_df, stat = ols.statistics_recursive()
print(f'PLS_(sep6)-QR({q}):', round(stat['out of sample'][1], 4))

s1 = []
for i in range(len(sector_list)):
    s1 += [i] * len(sector_list[i])
s2 = []
for i in range(len(sector_list)):
    s2 += range(len(sector_list[i]))
# p.34 圖14,15
for x, y in zip(s1, s2):
    var = sector_list[x][y]
    sector_list2 = sector_list[:x] + [sector_list[x][:y] + sector_list[x][y + 1:]] + sector_list[
                                                                                     x + 1:]  # 用切片做，移除sector_list[x][y]，不能用del或remove，會完全改變。
    ols2 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols2.seperate(sector_list2, dr_algo='fit_pls', n_components=1, oos_periods=oos_periods)
    ols2.fit_recursive(quantile=q)
    pred_df2, stat2 = ols2.statistics_recursive()
    print(var, ':', round(stat['out of sample'][1] - stat2['out of sample'][1], 4))
    stat_var_df = stat_var_df._append(
        {'var': var, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat2['out of sample'][1], 4)},
        ignore_index=True)
    if stat['out of sample'][1] - stat2['out of sample'][1] <= 0:
        keep_list[x].remove(var)
total_ = 0
for i in range(len(keep_list)):
    total_ += len(keep_list[i])
print(f'keep_list有{total_}個關鍵變數')  # 53個沒錯!!!
print('keep_list=', keep_list)

# p.36 圖17 類別的邊際貢獻: stat3-stat5  用切片做 沒錯!!!
for i in range(len(sector_list)):
    sector_list3 = sector_list[:i] + sector_list[i + 1:]  # keep_list刪掉就回不去了 用切片做
    ols5 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols5.seperate(sector_list3, dr_algo='fit_pls', n_components=1,
                  oos_periods=oos_periods)  # n_components=1 沒錯 從sector_list3的11類別 個別萃取出1個因子，共11個
    ols5.fit_recursive(quantile=q)
    pred_df5, stat5 = ols5.statistics_recursive()
    #print(stat5)
    print(i, ': Marginal PsuedoR2=', round(stat['out of sample'][1] - stat5['out of sample'][1], 4))
    print('----------------------------------------------------------------')
    stat_cat_df = stat_cat_df._append(
        {'cat': i, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat5['out of sample'][1], 4)},
        ignore_index=True)

print(stat_var_df)
print(stat_cat_df)
stat_var_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PLS_UP_6_KeyVar.csv"))
stat_cat_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PLS_UP_6_KeyCat.csv"))

PLS_(sep6)-QR(0.9): -0.319
MthCap : -0.0043
CAPEI : 0.0012
bm : -0.0021
evm : -0.0013
pe_op_basic : -0.0102
pe_op_dil : 0.0014
pe_exi : -0.01
pe_inc : 0.0021
ps : -0.0058
pcf : -0.0065
dpr : 0.0005
cfm : 0.0037
equity_invcap : -0.0073
debt_invcap : -0.0014
totdebt_invcap : -0.0005
capital_ratio : -0.0013
int_debt : -0.0137
int_totdebt : -0.0131
cash_lt : -0.0043
invt_act : -0.0013
rect_act : -0.0009
debt_at : 0.0004
debt_ebitda : -0.0106
short_debt : 0.0011
curr_debt : -0.0009
lt_debt : 0.0038
profit_lct : 0.0088
ocf_lct : -0.0006
cash_debt : 0.001
fcf_ocf : 0.0003
lt_ppent : 0.0004
dltt_be : -0.0038
debt_assets : -0.0001
debt_capital : -0.0047
de_ratio : -0.0029
intcov : -0.0046
intcov_ratio : -0.0027
ptb : 0.003
PEG_trailing : -0.0137
divyield : 0.0024
AM : -0.0025
AssetGrowth : -0.0044
BM : -0.001
BMdec : 0.0047
BookLeverage : 0.006
BPEBM : -0.0017
Cash : 0.0003
CF : 0.0004
cfp : -0.0
ChEQ : -0.0063
ChTax : 0.0025
CompEquIss : -0.0011
CompositeDebtIssuance : 0.007
ConvDebt : -0.0017

In [34]:
# HiTec PQR 上行(0.9) 4 因子
stat_var_df = pd.DataFrame(columns=['var', 'Marginal_PseudoR'])
stat_cat_df = pd.DataFrame(columns=['cat', 'Marginal_PseudoR'])

funda = financial + operating + supply + esg
macro = output_income + labor + housing + consump + money + intExRate + prices + stock
senti = firm_senti + market_senti
sector_list = [funda, technical, macro, senti]
keep_list = [funda.copy(), technical.copy(), macro.copy(), senti.copy()]

oos_periods = len(df) - 120
q = 0.9

ols = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
ols.seperate(sector_list, dr_algo='fit_pqr', n_components=1, oos_periods=oos_periods, quantile=q)
ols.fit_recursive(quantile=q)
pred_df, stat = ols.statistics_recursive()
print(f'PQR_(sep4)-QR({q}):', round(stat['out of sample'][1], 4))

s1 = []
for i in range(len(sector_list)):
    s1 += [i] * len(sector_list[i])
s2 = []
for i in range(len(sector_list)):
    s2 += range(len(sector_list[i]))
# p.34 圖14,15
for x, y in zip(s1, s2):
    var = sector_list[x][y]
    sector_list2 = sector_list[:x] + [sector_list[x][:y] + sector_list[x][y + 1:]] + sector_list[
                                                                                     x + 1:]  # 用切片做，移除sector_list[x][y]，不能用del或remove，會完全改變。
    ols2 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols2.seperate(sector_list2, dr_algo='fit_pqr', n_components=1, oos_periods=oos_periods, quantile=q)
    ols2.fit_recursive(quantile=q)
    pred_df2, stat2 = ols2.statistics_recursive()
    print(var, ':', round(stat['out of sample'][1] - stat2['out of sample'][1], 4))
    stat_var_df = stat_var_df._append(
        {'var': var, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat2['out of sample'][1], 4)},
        ignore_index=True)
    if stat['out of sample'][1] - stat2['out of sample'][1] <= 0:
        keep_list[x].remove(var)
total_ = 0
for i in range(len(keep_list)):
    total_ += len(keep_list[i])
print(f'keep_list有{total_}個關鍵變數')  # 53個沒錯!!!
print('keep_list=', keep_list)

# p.36 圖17 類別的邊際貢獻: stat3-stat5  用切片做 沒錯!!!
for i in range(len(sector_list)):
    sector_list3 = sector_list[:i] + sector_list[i + 1:]  # keep_list刪掉就回不去了 用切片做
    ols5 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols5.seperate(sector_list3, dr_algo='fit_pqr', n_components=1, oos_periods=oos_periods,
                  quantile=q)  # n_components=1 沒錯 從sector_list3的11類別 個別萃取出1個因子，共11個
    ols5.fit_recursive(quantile=q)
    pred_df5, stat5 = ols5.statistics_recursive()
    #print(stat5)
    print(i, ': Marginal PsuedoR2=', round(stat['out of sample'][1] - stat5['out of sample'][1], 4))
    print('----------------------------------------------------------------')
    stat_cat_df = stat_cat_df._append(
        {'cat': i, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat5['out of sample'][1], 4)},
        ignore_index=True)

print(stat_var_df)
print(stat_cat_df)
stat_var_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PQR_UP_4_KeyVar.csv"))
stat_cat_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PQR_UP_4_KeyCat.csv"))

PQR_(sep4)-QR(0.9): 0.0687
MthCap : 0.0014
CAPEI : 0.0023
bm : -0.0003
evm : -0.0022
pe_op_basic : 0.0023
pe_op_dil : -0.0033
pe_exi : -0.0027
pe_inc : 0.0
ps : -0.0017
pcf : -0.0016
dpr : -0.0004
cfm : -0.0009
equity_invcap : -0.0037
debt_invcap : -0.0023
totdebt_invcap : -0.0031
capital_ratio : -0.0009
int_debt : -0.0017
int_totdebt : -0.0024
cash_lt : 0.0016
invt_act : 0.0001
rect_act : -0.0022
debt_at : -0.0002
debt_ebitda : -0.0021
short_debt : -0.0055
curr_debt : -0.0043
lt_debt : -0.005
profit_lct : 0.0006
ocf_lct : -0.0056
cash_debt : -0.0061
fcf_ocf : -0.0036
lt_ppent : -0.003
dltt_be : -0.0003
debt_assets : -0.0002
debt_capital : -0.0041
de_ratio : -0.0007
intcov : -0.0041
intcov_ratio : -0.0019
ptb : -0.0004
PEG_trailing : -0.0004
divyield : -0.0051
AM : -0.007
AssetGrowth : 0.0032
BM : -0.0073
BMdec : 0.0021
BookLeverage : 0.0023
BPEBM : -0.0025
Cash : 0.0004
CF : -0.0045
cfp : -0.0021
ChEQ : -0.0014
ChTax : -0.0033
CompEquIss : -0.0052
CompositeDebtIssuance : -0.0008
ConvD

In [35]:
# HiTec PQR 上行(0.9) 5 因子
stat_var_df = pd.DataFrame(columns=['var', 'Marginal_PseudoR'])
stat_cat_df = pd.DataFrame(columns=['cat', 'Marginal_PseudoR'])

macro = output_income + labor + housing + consump + money + intExRate + prices + stock
senti = firm_senti + market_senti
opera = operating + supply + esg
sector_list = [financial, opera, technical, macro, senti]
keep_list = [financial.copy(), opera.copy(), technical.copy(), macro.copy(), senti.copy()]

oos_periods = len(df) - 120
q = 0.9

ols = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
ols.seperate(sector_list, dr_algo='fit_pqr', n_components=1, oos_periods=oos_periods, quantile=q)
ols.fit_recursive(quantile=q)
pred_df, stat = ols.statistics_recursive()
print(f'PQR_(sep5)-QR({q}):', round(stat['out of sample'][1], 4))

s1 = []
for i in range(len(sector_list)):
    s1 += [i] * len(sector_list[i])
s2 = []
for i in range(len(sector_list)):
    s2 += range(len(sector_list[i]))
# p.34 圖14,15
for x, y in zip(s1, s2):
    var = sector_list[x][y]
    sector_list2 = sector_list[:x] + [sector_list[x][:y] + sector_list[x][y + 1:]] + sector_list[
                                                                                     x + 1:]  # 用切片做，移除sector_list[x][y]，不能用del或remove，會完全改變。
    ols2 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols2.seperate(sector_list2, dr_algo='fit_pqr', n_components=1, oos_periods=oos_periods, quantile=q)
    ols2.fit_recursive(quantile=q)
    pred_df2, stat2 = ols2.statistics_recursive()
    print(var, ':', round(stat['out of sample'][1] - stat2['out of sample'][1], 4))
    stat_var_df = stat_var_df._append(
        {'var': var, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat2['out of sample'][1], 4)},
        ignore_index=True)
    if stat['out of sample'][1] - stat2['out of sample'][1] <= 0:
        keep_list[x].remove(var)
total_ = 0
for i in range(len(keep_list)):
    total_ += len(keep_list[i])
print(f'keep_list有{total_}個關鍵變數')  # 53個沒錯!!!
print('keep_list=', keep_list)

# p.36 圖17 類別的邊際貢獻: stat3-stat5  用切片做 沒錯!!!
for i in range(len(sector_list)):
    sector_list3 = sector_list[:i] + sector_list[i + 1:]  # keep_list刪掉就回不去了 用切片做
    ols5 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols5.seperate(sector_list3, dr_algo='fit_pqr', n_components=1, oos_periods=oos_periods,
                  quantile=q)  # n_components=1 沒錯 從sector_list3的11類別 個別萃取出1個因子，共11個
    ols5.fit_recursive(quantile=q)
    pred_df5, stat5 = ols5.statistics_recursive()
    #print(stat5)
    print(i, ': Marginal PsuedoR2=', round(stat['out of sample'][1] - stat5['out of sample'][1], 4))
    print('----------------------------------------------------------------')
    stat_cat_df = stat_cat_df._append(
        {'cat': i, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat5['out of sample'][1], 4)},
        ignore_index=True)

print(stat_var_df)
print(stat_cat_df)
stat_var_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PQR_UP_5_KeyVar.csv"))
stat_cat_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PQR_UP_5_KeyCat.csv"))

PQR_(sep5)-QR(0.9): 0.0636
MthCap : 0.0027
CAPEI : -0.0045
bm : 0.0003
evm : 0.003
pe_op_basic : -0.0038
pe_op_dil : -0.0025
pe_exi : 0.0009
pe_inc : -0.002
ps : -0.0012
pcf : -0.0031
dpr : -0.002
cfm : -0.0017
equity_invcap : 0.0016
debt_invcap : -0.0002
totdebt_invcap : 0.0006
capital_ratio : -0.0015
int_debt : 0.0021
int_totdebt : 0.0011
cash_lt : 0.0015
invt_act : 0.0007
rect_act : 0.0013
debt_at : -0.0019
debt_ebitda : -0.0045
short_debt : -0.0016
curr_debt : -0.0018
lt_debt : -0.0019
profit_lct : 0.0002
ocf_lct : -0.001
cash_debt : 0.0007
fcf_ocf : 0.002
lt_ppent : 0.0027
dltt_be : 0.001
debt_assets : 0.0
debt_capital : 0.0003
de_ratio : -0.0011
intcov : 0.0005
intcov_ratio : -0.0005
ptb : -0.0009
PEG_trailing : -0.0004
divyield : -0.0003
AM : -0.0003
AssetGrowth : 0.0029
BM : -0.0005
BMdec : -0.0002
BookLeverage : 0.0005
BPEBM : -0.0023
Cash : 0.0029
CF : -0.0011
cfp : 0.0002
ChEQ : 0.0006
ChTax : -0.0017
CompEquIss : -0.0024
CompositeDebtIssuance : -0.0018
ConvDebt : -0.0023
De

In [36]:
# HiTec PQR 上行(0.9) 6 因子
stat_var_df = pd.DataFrame(columns=['var', 'Marginal_PseudoR'])
stat_cat_df = pd.DataFrame(columns=['cat', 'Marginal_PseudoR'])

macro = output_income + labor + housing + consump + money + intExRate + prices + stock
senti = firm_senti + market_senti
opera = operating + supply
sector_list = [financial, opera, technical, esg, macro, senti]
keep_list = [financial.copy(), opera.copy(), technical.copy(), esg.copy(), macro.copy(), senti.copy()]

oos_periods = len(df) - 120
q = 0.9

ols = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
ols.seperate(sector_list, dr_algo='fit_pqr', n_components=1, oos_periods=oos_periods, quantile=q)
ols.fit_recursive(quantile=q)
pred_df, stat = ols.statistics_recursive()
print(f'PQR_(sep6)-QR({q}):', round(stat['out of sample'][1], 4))

s1 = []
for i in range(len(sector_list)):
    s1 += [i] * len(sector_list[i])
s2 = []
for i in range(len(sector_list)):
    s2 += range(len(sector_list[i]))
# p.34 圖14,15
for x, y in zip(s1, s2):
    var = sector_list[x][y]
    sector_list2 = sector_list[:x] + [sector_list[x][:y] + sector_list[x][y + 1:]] + sector_list[
                                                                                     x + 1:]  # 用切片做，移除sector_list[x][y]，不能用del或remove，會完全改變。
    ols2 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols2.seperate(sector_list2, dr_algo='fit_pqr', n_components=1, oos_periods=oos_periods, quantile=q)
    ols2.fit_recursive(quantile=q)
    pred_df2, stat2 = ols2.statistics_recursive()
    print(var, ':', round(stat['out of sample'][1] - stat2['out of sample'][1], 4))
    stat_var_df = stat_var_df._append(
        {'var': var, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat2['out of sample'][1], 4)},
        ignore_index=True)
    if stat['out of sample'][1] - stat2['out of sample'][1] <= 0:
        keep_list[x].remove(var)
total_ = 0
for i in range(len(keep_list)):
    total_ += len(keep_list[i])
print(f'keep_list有{total_}個關鍵變數')  # 53個沒錯!!!
print('keep_list=', keep_list)

# p.36 圖17 類別的邊際貢獻: stat3-stat5  用切片做 沒錯!!!
for i in range(len(sector_list)):
    sector_list3 = sector_list[:i] + sector_list[i + 1:]  # keep_list刪掉就回不去了 用切片做
    ols5 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols5.seperate(sector_list3, dr_algo='fit_pqr', n_components=1, oos_periods=oos_periods,
                  quantile=q)  # n_components=1 沒錯 從sector_list3的11類別 個別萃取出1個因子，共11個
    ols5.fit_recursive(quantile=q)
    pred_df5, stat5 = ols5.statistics_recursive()
    #print(stat5)
    print(i, ': Marginal PsuedoR2=', round(stat['out of sample'][1] - stat5['out of sample'][1], 4))
    print('----------------------------------------------------------------')
    stat_cat_df = stat_cat_df._append(
        {'cat': i, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat5['out of sample'][1], 4)},
        ignore_index=True)

print(stat_var_df)
print(stat_cat_df)
stat_var_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PQR_UP_6_KeyVar.csv"))
stat_cat_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PQR_UP_6_KeyCat.csv"))

PQR_(sep6)-QR(0.9): 0.0578
MthCap : 0.0017
CAPEI : -0.0006
bm : 0.0008
evm : 0.0058
pe_op_basic : -0.0012
pe_op_dil : -0.0033
pe_exi : -0.0029
pe_inc : -0.0046
ps : 0.005
pcf : -0.0011
dpr : -0.0
cfm : -0.003
equity_invcap : 0.0032
debt_invcap : -0.001
totdebt_invcap : 0.0007
capital_ratio : 0.0002
int_debt : 0.0043
int_totdebt : 0.0043
cash_lt : 0.004
invt_act : 0.0026
rect_act : 0.0022
debt_at : 0.0005
debt_ebitda : -0.0038
short_debt : -0.0069
curr_debt : -0.0004
lt_debt : 0.0021
profit_lct : 0.0006
ocf_lct : 0.0033
cash_debt : 0.0025
fcf_ocf : 0.002
lt_ppent : 0.0001
dltt_be : 0.0014
debt_assets : -0.0002
debt_capital : -0.0002
de_ratio : -0.0062
intcov : 0.0034
intcov_ratio : 0.0021
ptb : 0.0007
PEG_trailing : 0.0004
divyield : -0.0016
AM : 0.0016
AssetGrowth : 0.0067
BM : 0.0008
BMdec : 0.0017
BookLeverage : 0.0022
BPEBM : -0.0037
Cash : 0.0048
CF : 0.0011
cfp : 0.0024
ChEQ : -0.0004
ChTax : 0.0013
CompEquIss : -0.0045
CompositeDebtIssuance : 0.0003
ConvDebt : 0.0003
DebtIssuance

In [ ]:
# HiTec PQR 上行(0.9) 7 因子
stat_var_df = pd.DataFrame(columns=['var', 'Marginal_PseudoR'])
stat_cat_df = pd.DataFrame(columns=['cat', 'Marginal_PseudoR'])

macro = output_income + labor + housing + consump + money + intExRate + prices + stock
senti = firm_senti + market_senti
sector_list = [financial, operating, technical, supply, esg, macro, senti]
keep_list = [financial.copy(), operating.copy(), technical.copy(), supply.copy(), esg.copy(), macro.copy(),
             senti.copy()]

oos_periods = len(df) - 120
q = 0.9

ols = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
ols.seperate(sector_list, dr_algo='fit_pqr', n_components=1, oos_periods=oos_periods, quantile=q)
ols.fit_recursive(quantile=q)
pred_df, stat = ols.statistics_recursive()
print(f'PQR_(sep7)-QR({q}):', round(stat['out of sample'][1], 4))

s1 = []
for i in range(len(sector_list)):
    s1 += [i] * len(sector_list[i])
s2 = []
for i in range(len(sector_list)):
    s2 += range(len(sector_list[i]))
# p.34 圖14,15
for x, y in zip(s1, s2):
    var = sector_list[x][y]
    sector_list2 = sector_list[:x] + [sector_list[x][:y] + sector_list[x][y + 1:]] + sector_list[
                                                                                     x + 1:]  # 用切片做，移除sector_list[x][y]，不能用del或remove，會完全改變。
    ols2 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols2.seperate(sector_list2, dr_algo='fit_pqr', n_components=1, oos_periods=oos_periods, quantile=q)
    ols2.fit_recursive(quantile=q)
    pred_df2, stat2 = ols2.statistics_recursive()
    print(var, ':', round(stat['out of sample'][1] - stat2['out of sample'][1], 4))
    stat_var_df = stat_var_df._append(
        {'var': var, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat2['out of sample'][1], 4)},
        ignore_index=True)
    if stat['out of sample'][1] - stat2['out of sample'][1] <= 0:
        keep_list[x].remove(var)
total_ = 0
for i in range(len(keep_list)):
    total_ += len(keep_list[i])
print(f'keep_list有{total_}個關鍵變數')  # 53個沒錯!!!
print('keep_list=', keep_list)

# p.36 圖17 類別的邊際貢獻: stat3-stat5  用切片做 沒錯!!!
for i in range(len(sector_list)):
    sector_list3 = sector_list[:i] + sector_list[i + 1:]  # keep_list刪掉就回不去了 用切片做
    ols5 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols5.seperate(sector_list3, dr_algo='fit_pqr', n_components=1, oos_periods=oos_periods,
                  quantile=q)  # n_components=1 沒錯 從sector_list3的11類別 個別萃取出1個因子，共11個
    ols5.fit_recursive(quantile=q)
    pred_df5, stat5 = ols5.statistics_recursive()
    #print(stat5)
    print(i, ': Marginal PsuedoR2=', round(stat['out of sample'][1] - stat5['out of sample'][1], 4))
    print('----------------------------------------------------------------')
    stat_cat_df = stat_cat_df._append(
        {'cat': i, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat5['out of sample'][1], 4)},
        ignore_index=True)

print(stat_var_df)
print(stat_cat_df)
stat_var_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PQR_UP_7_KeyVar.csv"))
stat_cat_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PQR_UP_7_KeyCat.csv"))

PQR_(sep7)-QR(0.9): 0.011
MthCap : -0.0033
CAPEI : -0.0031
bm : -0.0009
evm : 0.0024
pe_op_basic : -0.003


In [ ]:
# HiTec PQR 上行(0.9) 15 因子
stat_var_df = pd.DataFrame(columns=['var', 'Marginal_PseudoR'])
stat_cat_df = pd.DataFrame(columns=['cat', 'Marginal_PseudoR'])

sector_list = [financial, supply, firm_senti, technical, operating, esg, output_income, labor, housing, consump, money,
               intExRate, prices, stock, market_senti]
keep_list = [financial.copy(), supply.copy(), firm_senti.copy(), technical.copy(), operating.copy(), esg.copy(),
             output_income.copy(), labor.copy(), housing.copy(), consump.copy(), money.copy(), intExRate.copy(),
             prices.copy(), stock.copy(), market_senti.copy()]

oos_periods = len(df) - 120
q = 0.9

ols = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
ols.seperate(sector_list, dr_algo='fit_pqr', n_components=1, oos_periods=oos_periods, quantile=q)
ols.fit_recursive(quantile=q)
pred_df, stat = ols.statistics_recursive()
print(f'PQR_(sep15)-QR({q}):', round(stat['out of sample'][1], 4))

s1 = []
for i in range(len(sector_list)):
    s1 += [i] * len(sector_list[i])
s2 = []
for i in range(len(sector_list)):
    s2 += range(len(sector_list[i]))
# p.34 圖14,15
for x, y in zip(s1, s2):
    var = sector_list[x][y]
    sector_list2 = sector_list[:x] + [sector_list[x][:y] + sector_list[x][y + 1:]] + sector_list[
                                                                                     x + 1:]  # 用切片做，移除sector_list[x][y]，不能用del或remove，會完全改變。
    ols2 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols2.seperate(sector_list2, dr_algo='fit_pqr', n_components=1, oos_periods=oos_periods, quantile=q)
    ols2.fit_recursive(quantile=q)
    pred_df2, stat2 = ols2.statistics_recursive()
    print(var, ':', round(stat['out of sample'][1] - stat2['out of sample'][1], 4))
    stat_var_df = stat_var_df._append(
        {'var': var, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat2['out of sample'][1], 4)},
        ignore_index=True)
    if stat['out of sample'][1] - stat2['out of sample'][1] <= 0:
        keep_list[x].remove(var)
total_ = 0
for i in range(len(keep_list)):
    total_ += len(keep_list[i])
print(f'keep_list有{total_}個關鍵變數')  # 53個沒錯!!!
print('keep_list=', keep_list)

# p.36 圖17 類別的邊際貢獻: stat3-stat5  用切片做 沒錯!!!
for i in range(len(sector_list)):
    sector_list3 = sector_list[:i] + sector_list[i + 1:]  # keep_list刪掉就回不去了 用切片做
    ols5 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols5.seperate(sector_list3, dr_algo='fit_pqr', n_components=1, oos_periods=oos_periods,
                  quantile=q)  # n_components=1 沒錯 從sector_list3的11類別 個別萃取出1個因子，共11個
    ols5.fit_recursive(quantile=q)
    pred_df5, stat5 = ols5.statistics_recursive()
    #print(stat5)
    print(i, ': Marginal PsuedoR2=', round(stat['out of sample'][1] - stat5['out of sample'][1], 4))
    print('----------------------------------------------------------------')
    stat_cat_df = stat_cat_df._append(
        {'cat': i, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat5['out of sample'][1], 4)},
        ignore_index=True)

print(stat_var_df)
print(stat_cat_df)
stat_var_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PQR_UP_15_KeyVar.csv"))
stat_cat_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PQR_UP_15_KeyCat.csv"))

In [ ]:
# HiTec PLS 下行(0.1) 5 因子
stat_var_df = pd.DataFrame(columns=['var', 'Marginal_PseudoR'])
stat_cat_df = pd.DataFrame(columns=['cat', 'Marginal_PseudoR'])

macro = output_income + labor + housing + consump + money + intExRate + prices + stock
senti = firm_senti + market_senti
opera = operating + supply + esg
sector_list = [financial, opera, technical, macro, senti]
keep_list = [financial.copy(), opera.copy(), technical.copy(), macro.copy(), senti.copy()]

oos_periods = len(df) - 120
q = 0.1

ols = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
ols.seperate(sector_list, dr_algo='fit_pls', n_components=1, oos_periods=oos_periods)
ols.fit_recursive(quantile=q)
pred_df, stat = ols.statistics_recursive()
print(f'PLS_(sep6)-QR({q}):', round(stat['out of sample'][1], 4))

s1 = []
for i in range(len(sector_list)):
    s1 += [i] * len(sector_list[i])
s2 = []
for i in range(len(sector_list)):
    s2 += range(len(sector_list[i]))
# p.34 圖14,15
for x, y in zip(s1, s2):
    var = sector_list[x][y]
    sector_list2 = sector_list[:x] + [sector_list[x][:y] + sector_list[x][y + 1:]] + sector_list[
                                                                                     x + 1:]  # 用切片做，移除sector_list[x][y]，不能用del或remove，會完全改變。
    ols2 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols2.seperate(sector_list2, dr_algo='fit_pls', n_components=1, oos_periods=oos_periods)
    ols2.fit_recursive(quantile=q)
    pred_df2, stat2 = ols2.statistics_recursive()
    print(var, ':', round(stat['out of sample'][1] - stat2['out of sample'][1], 4))
    stat_var_df = stat_var_df._append(
        {'var': var, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat2['out of sample'][1], 4)},
        ignore_index=True)
    if stat['out of sample'][1] - stat2['out of sample'][1] <= 0:
        keep_list[x].remove(var)
total_ = 0
for i in range(len(keep_list)):
    total_ += len(keep_list[i])
print(f'keep_list有{total_}個關鍵變數')  # 53個沒錯!!!
print('keep_list=', keep_list)

# p.36 圖17 類別的邊際貢獻: stat3-stat5  用切片做 沒錯!!!
for i in range(len(sector_list)):
    sector_list3 = sector_list[:i] + sector_list[i + 1:]  # keep_list刪掉就回不去了 用切片做
    ols5 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols5.seperate(sector_list3, dr_algo='fit_pls', n_components=1,
                  oos_periods=oos_periods)  # n_components=1 沒錯 從sector_list3的11類別 個別萃取出1個因子，共11個
    ols5.fit_recursive(quantile=q)
    pred_df5, stat5 = ols5.statistics_recursive()
    #print(stat5)
    print(i, ': Marginal PsuedoR2=', round(stat['out of sample'][1] - stat5['out of sample'][1], 4))
    print('----------------------------------------------------------------')
    stat_cat_df = stat_cat_df._append(
        {'cat': i, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat5['out of sample'][1], 4)},
        ignore_index=True)

print(stat_var_df)
print(stat_cat_df)
stat_var_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PLS_down_5_KeyVar.csv"))
stat_cat_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PLS_down_5_KeyCat.csv"))

In [ ]:
# HiTec PLS 下行(0.1) 6 因子
stat_var_df = pd.DataFrame(columns=['var', 'Marginal_PseudoR'])
stat_cat_df = pd.DataFrame(columns=['cat', 'Marginal_PseudoR'])

macro = output_income + labor + housing + consump + money + intExRate + prices + stock
senti = firm_senti + market_senti
opera = operating + supply
sector_list = [financial, opera, technical, esg, macro, senti]
keep_list = [financial.copy(), opera.copy(), technical.copy(), esg.copy(), macro.copy(), senti.copy()]

oos_periods = len(df) - 120
q = 0.1

ols = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
ols.seperate(sector_list, dr_algo='fit_pls', n_components=1, oos_periods=oos_periods)
ols.fit_recursive(quantile=q)
pred_df, stat = ols.statistics_recursive()
print(f'PLS_(sep6)-QR({q}):', round(stat['out of sample'][1], 4))

s1 = []
for i in range(len(sector_list)):
    s1 += [i] * len(sector_list[i])
s2 = []
for i in range(len(sector_list)):
    s2 += range(len(sector_list[i]))
# p.34 圖14,15
for x, y in zip(s1, s2):
    var = sector_list[x][y]
    sector_list2 = sector_list[:x] + [sector_list[x][:y] + sector_list[x][y + 1:]] + sector_list[
                                                                                     x + 1:]  # 用切片做，移除sector_list[x][y]，不能用del或remove，會完全改變。
    ols2 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols2.seperate(sector_list2, dr_algo='fit_pls', n_components=1, oos_periods=oos_periods)
    ols2.fit_recursive(quantile=q)
    pred_df2, stat2 = ols2.statistics_recursive()
    print(var, ':', round(stat['out of sample'][1] - stat2['out of sample'][1], 4))
    stat_var_df = stat_var_df._append(
        {'var': var, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat2['out of sample'][1], 4)},
        ignore_index=True)
    if stat['out of sample'][1] - stat2['out of sample'][1] <= 0:
        keep_list[x].remove(var)
total_ = 0
for i in range(len(keep_list)):
    total_ += len(keep_list[i])
print(f'keep_list有{total_}個關鍵變數')  # 53個沒錯!!!
print('keep_list=', keep_list)

# p.36 圖17 類別的邊際貢獻: stat3-stat5  用切片做 沒錯!!!
for i in range(len(sector_list)):
    sector_list3 = sector_list[:i] + sector_list[i + 1:]  # keep_list刪掉就回不去了 用切片做
    ols5 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols5.seperate(sector_list3, dr_algo='fit_pls', n_components=1,
                  oos_periods=oos_periods)  # n_components=1 沒錯 從sector_list3的11類別 個別萃取出1個因子，共11個
    ols5.fit_recursive(quantile=q)
    pred_df5, stat5 = ols5.statistics_recursive()
    #print(stat5)
    print(i, ': Marginal PsuedoR2=', round(stat['out of sample'][1] - stat5['out of sample'][1], 4))
    print('----------------------------------------------------------------')
    stat_cat_df = stat_cat_df._append(
        {'cat': i, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat5['out of sample'][1], 4)},
        ignore_index=True)

print(stat_var_df)
print(stat_cat_df)
stat_var_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PLS_down_6_KeyVar.csv"))
stat_cat_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PLS_down_6_KeyCat.csv"))

In [ ]:
# HiTec PLS 下行(0.1) 7 因子
stat_var_df = pd.DataFrame(columns=['var', 'Marginal_PseudoR'])
stat_cat_df = pd.DataFrame(columns=['cat', 'Marginal_PseudoR'])

macro = output_income + labor + housing + consump + money + intExRate + prices + stock
senti = firm_senti + market_senti
sector_list = [financial, operating, technical, supply, esg, macro, senti]
keep_list = [financial.copy(), operating.copy(), technical.copy(), supply.copy(), esg.copy(), macro.copy(),
             senti.copy()]

oos_periods = len(df) - 120
q = 0.1

ols = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
ols.seperate(sector_list, dr_algo='fit_pls', n_components=1, oos_periods=oos_periods)
ols.fit_recursive(quantile=q)
pred_df, stat = ols.statistics_recursive()
print(f'PLS_(sep7)-QR({q}):', round(stat['out of sample'][1], 4))

s1 = []
for i in range(len(sector_list)):
    s1 += [i] * len(sector_list[i])
s2 = []
for i in range(len(sector_list)):
    s2 += range(len(sector_list[i]))
# p.34 圖14,15
for x, y in zip(s1, s2):
    var = sector_list[x][y]
    sector_list2 = sector_list[:x] + [sector_list[x][:y] + sector_list[x][y + 1:]] + sector_list[
                                                                                     x + 1:]  # 用切片做，移除sector_list[x][y]，不能用del或remove，會完全改變。
    ols2 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols2.seperate(sector_list2, dr_algo='fit_pls', n_components=1, oos_periods=oos_periods)
    ols2.fit_recursive(quantile=q)
    pred_df2, stat2 = ols2.statistics_recursive()
    print(var, ':', round(stat['out of sample'][1] - stat2['out of sample'][1], 4))
    stat_var_df = stat_var_df._append(
        {'var': var, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat2['out of sample'][1], 4)},
        ignore_index=True)
    if stat['out of sample'][1] - stat2['out of sample'][1] <= 0:
        keep_list[x].remove(var)
total_ = 0
for i in range(len(keep_list)):
    total_ += len(keep_list[i])
print(f'keep_list有{total_}個關鍵變數')  # 53個沒錯!!!
print('keep_list=', keep_list)

# p.36 圖17 類別的邊際貢獻: stat3-stat5  用切片做 沒錯!!!
for i in range(len(sector_list)):
    sector_list3 = sector_list[:i] + sector_list[i + 1:]  # keep_list刪掉就回不去了 用切片做
    ols5 = ModuleQR(df, 'MthRet', rescale_data=True, oos_periods=oos_periods)
    ols5.seperate(sector_list3, dr_algo='fit_pls', n_components=1,
                  oos_periods=oos_periods)  # n_components=1 沒錯 從sector_list3的11類別 個別萃取出1個因子，共11個
    ols5.fit_recursive(quantile=q)
    pred_df5, stat5 = ols5.statistics_recursive()
    #print(stat5)
    print(i, ': Marginal PsuedoR2=', round(stat['out of sample'][1] - stat5['out of sample'][1], 4))
    print('----------------------------------------------------------------')
    stat_cat_df = stat_cat_df._append(
        {'cat': i, 'Marginal_PseudoR': round(stat['out of sample'][1] - stat5['out of sample'][1], 4)},
        ignore_index=True)

print(stat_var_df)
print(stat_cat_df)
stat_var_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PLS_down_7_KeyVar.csv"))
stat_cat_df.to_csv(str(config.path / f"thesis_code/result_657/{industry_}_PLS_down_7_KeyCat.csv"))
